In [1]:
import dask.dataframe as dd


In [7]:
ddf = dd.read_csv('/data2st1/junyi/snmc_bedgraph/merged/all_unified.bedGraph',sep='\t',header=0, assume_missing=True)

In [8]:
ddf

,chrom,start,end,ABC_NN.CGN,ACB-BST-FS_D1_Gaba.CGN,ADP-MPO_Trp73_Glut.CGN,AD_Serpinb7_Glut.CGN,AHN_Onecut3_Gaba.CGN,AHN-RCH-LHA_Otp_Fezf1_Glut.CGN,AHN-SBPV-PVHd_Pdrm12_Gaba.CGN,APN_C1ql2_Glut.CGN,APN_C1ql4_Glut.CGN,ARH-PVp_Tbx3_Gaba.CGN,ARH-PVp_Tbx3_Glut.CGN,Astro-NT_NN.CGN,Astro-TE_NN.CGN,AV_Col27a1_Glut.CGN,AVPV-MEPO-SFO_Tbr1_Glut.CGN,Bergmann_NN.CGN,B-PB_Nr4a2_Glut.CGN,BST-MPN_Six3_Nrgn_Gaba.CGN,BST-po_Iigp1_Glut.CGN,BST-SI-AAA_Six3_Slc22a3_Gaba.CGN,BST_Tac2_Gaba.CGN,CA1-ProS_Glut.CGN,CA2-FC-IG_Glut.CGN,CA3_Glut.CGN,CB_Granule_Glut.CGN,CBN_Neurod2_Pvalb_Glut.CGN,CB_PLI_Gly-Gaba.CGN,CBX_Golgi_Gly-Gaba.CGN,CBX_MLI_Cdh22_Gaba.CGN,CBX_MLI_Megf11_Gaba.CGN,CBX_Purkinje_Gaba.CGN,CEA-AAA-BST_Six3_Sp9_Gaba.CGN,CEA-BST_Ebf1_Pdyn_Gaba.CGN,CEA-BST_Gal_Avp_Gaba.CGN,CEA-BST_Rai14_Pdyn_Crh_Gaba.CGN,CEA-BST_Six3_Cyp26b1_Gaba.CGN,CHOR_NN.CGN,CLA-EPd-CTX_Car3_Glut.CGN,CM-IAD-CL-PCN_Sema5b_Glut.CGN,COAa-PAA-MEA_Barhl2_Glut.CGN,COAp_Grxcr2_Glut.CGN,CS-PRNr-DR_En1_Sox2_Gaba.CGN,CS-PRNr-PCG_Tmem163_Otp_Gaba.CGN,CS-RPO_Meis2_Gaba.CGN,CT_SUB_Glut.CGN,CU-ECU-SPVI_Foxb1_Glut.CGN,CUN_Evx2_Lhx2_Glut.CGN,CUN-PPN_Evx2_Meis2_Glut.CGN,DCO_Il22_Gly-Gaba.CGN,DG_Glut.CGN,DG-PIR_Ex_IMN.CGN,DMH_Hmx2_Gaba.CGN,DMH_Hmx2_Glut.CGN,DMH-LHA_Gsx1_Gaba.CGN,DMH-LHA_Vgll2_Glut.CGN,DMH_Nkx2-4_Glut.CGN,DMH_Prdm13_Gaba.CGN,DMX_VII_Tbx20_Chol.CGN,DTN-LDT-IPN_Otp_Pax3_Gaba.CGN,Endo_NN.CGN,ENTmv-PA-COAp_Glut.CGN,GPe-SI_Sox6_Cyp26b1_Gaba.CGN,GPi_Tbr1_Cngb3_Gaba-Glut.CGN,HPF_CR_Glut.CGN,IA_Mgp_Gaba.CGN,IC_Six3_En2_Gaba.CGN,IC_Tfap2d_Maf_Glut.CGN,IF-RL-CLI-PAG_Foxa1_Glut.CGN,IO_Fgl2_Glut.CGN,IPN-LDT_Vsx2_Nkx6-1_Glut.CGN,IPN_Otp_Crisp1_Gaba.CGN,IT_AON-TT-DP_Glut.CGN,IT_EP-CLA_Glut.CGN,L2_3_IT_CTX_Glut.CGN,L2_3_IT_ENT_Glut.CGN,L2_3_IT_PIR-ENTl_Glut.CGN,L2_3_IT_PPP_Glut.CGN,L2_3_IT_RSP_Glut.CGN,L2_IT_ENT-po_Glut.CGN,L2_IT_PPP-APr_Glut.CGN,L4_5_IT_CTX_Glut.CGN,L4_RSP-ACA_Glut.CGN,L5_6_IT_TPE-ENT_Glut.CGN,L5_ET_CTX_Glut.CGN,L5_IT_CTX_Glut.CGN,L5_NP_CTX_Glut.CGN,L6b_CT_ENT_Glut.CGN,L6b_CTX_Glut.CGN,L6b_EPd_Glut.CGN,L6_CT_CTX_Glut.CGN,L6_IT_CTX_Glut.CGN,LA-BLA-BMA-PA_Glut.CGN,Lamp5_Gaba.CGN,Lamp5_Lhx6_Gaba.CGN,LDT-DTN_Gata3_Nfix_Gaba.CGN,LDT-PCG-CS_Gata3_Lhx1_Gaba.CGN,LDT-PCG_St18_Gaba.CGN,LDT-PCG_Vsx2_Lhx4_Glut.CGN,LDT_Vsx2_Nkx6-1_Nfib_Glut.CGN,LGv-SPFp-SPFm_Nkx2-2_Tcf7l2_Gaba.CGN,LGv-ZI_Otx2_Gaba.CGN,LHA-AHN-PVH_Otp_Trh_Glut.CGN,LHA_Barhl2_Glut.CGN,LHA-MEA_Otp_Glut.CGN,LHA_Pmch_Glut.CGN,LH_Pou4f1_Sox1_Glut.CGN,LSX_Nkx2-1_Gaba.CGN,LSX_Otx2_Gaba.CGN,LSX_Prdm12_Slit2_Gaba.CGN,LSX_Prdm12_Zeb2_Gaba.CGN,LSX_Sall3_Lmo1_Gaba.CGN,LSX_Sall3_Pax6_Gaba.CGN,MB-ant-ve_Dmrta2_Glut.CGN,MB-MY_Tph2_Glut-Sero.CGN,MDRN_Hoxb5_Ebf2_Gly-Gaba.CGN,MEA-BST_Lhx6_Nfib_Gaba.CGN,MEA-BST_Lhx6_Nr2e1_Gaba.CGN,MEA-BST_Lhx6_Sp9_Gaba.CGN,MEA-BST_Otp_Zic2_Glut.CGN,MEA-BST_Sox6_Gaba.CGN,MEA-COA-BMA_Ccdc42_Glut.CGN,MEA_Otp_Foxp2_Glut.CGN,MEA_Slc17a7_Glut.CGN,MG-POL-SGN_Nts_Glut.CGN,MH_Tac2_Glut.CGN,Microglia_NN.CGN,MM-ant_Foxb1_Glut.CGN,MPN-MPO-LPO_Lhx6_Zfhx3_Gaba.CGN,MPN-MPO-PVpo_Hmx2_Glut.CGN,MPO-ADP_Lhx8_Gaba.CGN,MRN_Pou3f1_C1ql4_Glut.CGN,MRN-VTN-PPN_Pax5_Cdh23_Gaba.CGN,MS-SF_Bsx_Glut.CGN,MV_Nkx6-1_Gly-Gaba.CGN,MV_Pax6_Gly-Gaba.CGN,MV-SPIV_Phox2b_Ebf3_Lbx1_Glut.CGN,MV-SPIV-PRP_Dmbx1_Gly-Gaba.CGN,MV-SPIV_Zic4_Neurod2_Glut.CGN,MY_Lhx1_Gly-Gaba.CGN,MY_Prox1_Lmo7_Gly-Gaba.CGN,NDB-SI-ant_Prdm12_Gaba.CGN,NDB-SI-MA-STRv_Lhx8_Gaba.CGN,ND-INC_Foxd2_Glut.CGN,NI-RPO_Gata3_Nr4a2_Gaba.CGN,NLL_Gata3_Gly-Gaba.CGN,NLL-SOC_Spp1_Glut.CGN,NP_SUB_Glut.CGN,NTS_Dbh_Glut.CGN,NTS-MDRNd_Prox1_Zic1_Gly-Gaba.CGN,NTS-PARN_Neurod2_Gly-Gaba.CGN,NTS_Phox2b_Glut.CGN,OB_Dopa-Gaba.CGN,OB_Eomes_Ms4a15_Glut.CGN,OB-in_Frmd7_Gaba.CGN,OB-out_Frmd7_Gaba.CGN,OB-STR-CTX_Inh_IMN.CGN,OB_Trdn_Gaba.CGN,Oligo_NN.CGN,OPC_NN.CGN,OT_D3_Folh1_Gaba.CGN,PAG-MRN_Pou3f1_Glut.CGN,PAG-MRN-RN_Foxa2_Gaba.CGN,PAG-MRN_Tfap2b_Glut.CGN,PAG-ND-PCG_Onecut1_Gaba.CGN,PAG_Pou4f1_Bnc2_Glut.CGN,PAG_Pou4f1_Ebf2_Glut.CGN,PAG_Pou4f2_Glut.CGN,PAG_Pou4f2_Mesi2_Glut.CGN,PAG_Pou4f3_Glut.CGN,PAG-PPN_Pax5_Sox21_Gaba.CGN,PAG-RN_Nkx2-2_Otx1_Gaba.CGN,PAG-SC_Neurod2_Meis2_Glut.CGN,PAG-SC_Pou4f1_Zic1_G

In [9]:
matrix = ddf.iloc[:,3:]

In [10]:
matrix = matrix.compute()

In [11]:
import scipy.sparse as sp
sparse_matrix = sp.csr_matrix(matrix.values)


In [13]:
import scanpy as sc
adata = sc.AnnData(sparse_matrix)

In [16]:
adata = adata.T

In [22]:
region_df = ddf.iloc[:,0:3]

In [24]:
region_df = region_df.compute()

In [26]:
region_df['start'] = region_df['start'].astype(int)
region_df['end'] = region_df['end'].astype(int)

In [27]:
region_df

,chrom,start,end
0,chr1,3000826,3000827
1,chr1,3000827,3000828
2,chr1,3001006,3001007
3,chr1,3001007,3001008
4,chr1,3001017,3001018
...,...,...,...
76479,chrY,90844638,90844639
76480,chrY,90844648,90844649
76481,chrY,90844649,90844650
76482,chrY,90844675,90844676


In [21]:
adata.obs_names = ddf.columns[3:]

In [29]:
adata.var = region_df

/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/anndata.py:747: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [32]:
adata.var.index = adata.var['chrom'] + ':' + adata.var['start'].astype(str) + '-' + adata.var['end'].astype(str)

In [35]:
adata.var.index = adata.var.index.astype(str)
adata.obs.index = adata.obs.index.astype(str)

In [36]:
adata.write_h5ad('/data2st1/junyi/snmc_bedgraph/merged/meth_frac_all.h5ad')

In [39]:
mapping = {
    'Oligo_NN': 'OPC-Oligo',
    'Microglia_NN': 'Immune',
    'Endo_NN': 'Vascular',
    'Astro-TE_NN': 'Astro-Epen',
    'Astro-OLF_NN': 'Astro-Epen',
    'Astro-NT_NN': 'Astro-Epen',
    'Astroependymal_NN': 'Astro-Epen',
    'Hypendymal_NN': 'Astro-Epen',
    'Ependymal_NN': 'Astro-Epen',
    'Tanycyte_NN': 'Astro-Epen',
    'Bergmann_NN': 'Astro-Epen',
    'CHOR_NN': 'Astro-Epen',
    'Astro-CB_NN': 'Astro-Epen',
    'OPC_NN': 'OPC-Oligo',
    'OEC_NN': 'Astro-Epen',
    'BAM_NN': 'Immune',
    'DC_NN': 'Immune',
    'Lymphoid_NN': 'Immune',
    'VLMC_NN': 'Vascular',
    'ABC_NN': 'Immune',
    'SMC_NN': 'Vascular',
    'Peri_NN': 'Vascular'
}

In [52]:
adata.obs['Subtype'] = adata.obs.index.str.replace('.CGN','')

In [53]:
adata.obs['Celltype.l1'] = adata.obs['Subtype'].map(mapping).fillna('Neuron')


In [58]:
adata.obs['Celltype.l2'] = adata.obs['Subtype'].str.split('_').str[0]+"_"+adata.obs['Subtype'].str.split('_').str[-1]

In [60]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)

: 